In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model
%matplotlib inline
plt.style.use('ggplot')


import seaborn as sns
sns.set(style="whitegrid", color_codes=True)
sns.set(font_scale=1)

In [2]:
#pandas settings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
train = pd.read_csv('/Users/rociobecerra/Desktop/Project-Tribore/RocioBecerra/train.csv')
test = pd.read_csv('/Users/rociobecerra/Desktop/Project-Tribore/RocioBecerra/test.csv')

In [5]:
###TRAIN DATA PREPROCESSING FULL MODEL###

In [6]:
#Check missing values 
check_columns=train.columns[train.isnull().any()]
train[check_columns].isnull().sum()

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [9]:
train = train.sort_index(axis=1)

In [10]:
#OLS to impute LotFrontage
def fill_mv_LotFrontage(df,r0):
    '''
    Fills in missing lot frontage values using regression where it makes sense
    and median value otherwise
    '''
    ols = linear_model.LinearRegression()
    nhoods = df.Neighborhood.unique()

    for nhood in nhoods:
        df_n = df[ df['Neighborhood'] == nhood ]
        mv_idx = (df['Neighborhood'] == nhood) & (df['LotFrontage'].isnull())
        
        # iwf there are mv in this neighborhood
        if np.sum(mv_idx) > 0:
            X = np.array(df_n.loc[ df_n['LotFrontage'].notnull(),['LotArea']]).reshape(-1,1)
            Y = np.array(df_n.loc[ df_n['LotFrontage'].notnull(), ['LotFrontage']])
            ols.fit(X,Y)
            R2 = ols.score(X,Y)
            #print(nhood, "R^2: %.2f" %R2, "beta_1: %.3f" %ols.coef_, "beta_0: %.3f" %ols.intercept_)
        
            # if neighborhood based regression on LotArea has decent R^2
            if R2 > r0:
                df.loc[ mv_idx , ['LotFrontage'] ] = ols.predict( np.array(df.loc[mv_idx, 'LotArea' ]).reshape(-1,1) )
                print("imputed with regression \n", df.loc[ mv_idx , ['LotFrontage'] ],"\n" )
            else:
                df.loc[ mv_idx , ['LotFrontage'] ] = np.median(X)
                print("imputed with neighborhood median \n",  df.loc[ mv_idx , ['LotFrontage'] ],"\n" )
    return df

In [11]:
train = fill_mv_LotFrontage(train, 0.5)

In [12]:
train['MSSubClass'] = train['MSSubClass'].astype(str) #MSSubClass: The building class CONVERT to String 
train['MoSold'] = train['MoSold'].astype(str) #MoSold: Month Sold
train["Electrical"] = train["Electrical"].fillna('SBrkr') #train["Electrical"] = train["Electrical"].fillna('SBrkr')
train['MasVnrArea'] = train['MasVnrArea'].replace(np.nan, 0)
train['GarageYrBlt'] = train['GarageYrBlt'].replace(np.nan, 0)

In [13]:
#Fill NaN with None 
categorical = ['Alley','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1', \
               'BsmtFinType2','FireplaceQu','GarageType','GarageFinish', \
               'GarageQual','GarageCond','PoolQC','Fence','MiscFeature']
c_list = sorted(categorical)
for feature in c_list:
    train[feature]=train[feature].replace(np.nan, 'None')

In [14]:
#Check missing values 
check_columns=train.columns[train.isnull().any()]
train[check_columns].isnull().sum()

Series([], dtype: float64)

In [16]:
#Dummy code
train = pd.get_dummies(train, drop_first=True)
train.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,Fireplaces,FullBath,GarageArea,GarageCars,GarageYrBlt,GrLivArea,HalfBath,Id,KitchenAbvGr,LotArea,LotFrontage,LowQualFinSF,MasVnrArea,MiscVal,OpenPorchSF,OverallCond,OverallQual,PoolArea,SalePrice,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,Alley_None,Alley_Pave,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_TA,CentralAir_Y,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageCond_Fa,GarageCond_Gd,GarageCond_None,GarageCond_Po,GarageCond_TA,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_None,GarageQual_Po,GarageQual_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,LandContour_HLS,LandContour_Low,LandContour_Lvl,LandSlope_Mod,LandSlope_Sev,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LotShape_IR2,LotShape_IR3,LotShape_Reg,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_10,MoSold_11,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,PoolQC_None,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,Roo

In [17]:
###TEST PREPROCESSING###

In [18]:
#Check Missing Values 
check_columns=test.columns[test.isnull().any()]
test[check_columns].isnull().sum()

MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        16
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
GarageCars         1
GarageArea         1
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
SaleType           1
dtype: int64

In [19]:
test = test.sort_index(axis=1)

In [20]:
test = fill_mv_LotFrontage(test, 0.5)
test['MSSubClass'] = test['MSSubClass'].astype(str) #MSSubClass: The building class CONVERT to String 
test['MoSold'] = test['MoSold'].astype(str)

imputed with neighborhood median 
       LotFrontage
52         9350.0
59         9350.0
187        9350.0
188        9350.0
199        9350.0
297        9350.0
298        9350.0
301        9350.0
308        9350.0
592        9350.0
604        9350.0
614        9350.0
819        9350.0
927        9350.0
929        9350.0
935        9350.0
936        9350.0
943        9350.0
963        9350.0
971        9350.0
1154       9350.0
1156       9350.0
1157       9350.0
1264       9350.0
1267       9350.0
1277       9350.0
1278       9350.0
1281       9350.0
1304       9350.0 

imputed with regression 
       LotFrontage
6       59.414833
167     73.505082
174     62.456878
229     63.055676
230     60.131913
231     77.567302
235     88.674647
476     85.503232
481     81.914136
485    145.316560
487    204.678941
528     59.525722
529     60.952489
532     58.564687
793     67.424688
794     61.843294
797     65.240182
798     58.213539
865     74.089095
867     62.224012
1160    78.121745
1

In [21]:
#Fill NaN with None 
categorical = ['Alley','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1', \
               'BsmtFinType2','FireplaceQu','GarageType','GarageFinish', \
               'GarageQual','GarageCond','PoolQC','Fence','MiscFeature']
c_list = sorted(categorical)
for feature in c_list:
    test[feature]=test[feature].replace(np.nan, 'None')

In [22]:
check_columns=test.columns[test.isnull().any()]
test[check_columns].isnull().sum()

BsmtFinSF1       1
BsmtFinSF2       1
BsmtFullBath     2
BsmtHalfBath     2
BsmtUnfSF        1
Exterior1st      1
Exterior2nd      1
Functional       2
GarageArea       1
GarageCars       1
GarageYrBlt     78
KitchenQual      1
MSZoning         4
MasVnrArea      15
SaleType         1
TotalBsmtSF      1
Utilities        2
dtype: int64

In [24]:
test[test.isnull().any(axis=1)]

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,CentralAir,Condition1,Condition2,Electrical,EnclosedPorch,ExterCond,ExterQual,Exterior1st,Exterior2nd,Fence,FireplaceQu,Fireplaces,Foundation,FullBath,Functional,GarageArea,GarageCars,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,GrLivArea,HalfBath,Heating,HeatingQC,HouseStyle,Id,KitchenAbvGr,KitchenQual,LandContour,LandSlope,LotArea,LotConfig,LotFrontage,LotShape,LowQualFinSF,MSSubClass,MSZoning,MasVnrArea,MasVnrType,MiscFeature,MiscVal,MoSold,Neighborhood,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,PoolQC,RoofMatl,RoofStyle,SaleCondition,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
53,1728,0,0,None,6,Duplex,TA,No,1500.0,0.0,BLQ,Unf,2.0,0.0,TA,228.0,Y,Norm,Norm,SBrkr,0,TA,TA,HdBoard,HdBoard,None,None,0,CBlock,2,Typ,0.0,0.0,None,None,None,None,NaN,1728,0,GasA,TA,1Story,1514,2,TA,Lvl,Gtl,13260,Inside,98.000000,IR1,0,90,RL,144.0,BrkFace,None,0,1,NAmes,0,6,5,Y,0,None,CompShg,Hip,Abnorml,Oth,0,Pave,10,1728.0,AllPub,0,1962,2001,2010
71,936,0,0,Grvl,2,1Fam,TA,No,579.0,0.0,BLQ,Unf,1.0,0.0,TA,357.0,Y,Artery,Norm,SBrkr,135,TA,TA,Wd Sdng,Wd Sdng,MnPrv,Gd,1,PConc,1,Typ,0.0,0.0,None,None,None,None,NaN,936,0,GasA,TA,1Story,1532,1,TA,Lvl,Gtl,4485,Inside,56.000000,Reg,0,30,RM,0.0,None,None,0,5,OldTown,0,7,5,P,0,None,CompShg,Gable,Normal,WD,0,Pave,5,936.0,AllPub,51,1920,1950,2010
79,1128,1128,0,None,4,2fmCon,TA,Mn,0.0,0.0,Unf,Unf,0.0,0.0,TA,840.0,N,Norm,Norm,FuseF,18,Fa,TA,MetalSd,MetalSd,None,None,0,BrkTil,2,Typ,0.0,0.0,None,None,None,None,NaN,2256,0,Grav,Fa,2Story,1540,2,Fa,Lvl,Gtl,9045,Inside,100.000000,Reg,0,190,RM,0.0,None,None,0,6,OldTown,18,3,5,N,0,None,CompShg,Gable,Abnorml,WD,0,Pave,12,840.0,AllPub,0,1910,1950,2010
92,780,780,0,Grvl,3,1Fam,TA,No,0.0,0.0,Unf,Unf,0.0,0.0,TA,780.0,Y,Norm,Norm,SBrkr,0,Gd,Gd,Wd Sdng,Wd Sdng,None,None,0,PConc,1,Typ,0.0,0.0,None,None,None,None,NaN,1560,1,GasA,Ex,2Story,1553,1,Gd,Lvl,Gtl,12900,Inside,60.000000,Reg,0,70,RM,0.0,None,None,0,5,OldTown,0,8,6,N,0,None,CompShg,Gable,Normal,WD,168,Pave,7,780.0,AllPub,344,1912,2009,2010
95,725,499,0,None,3,1Fam,Fa,No,0.0,0.0,Unf,Unf,0.0,0.0,Gd,689.0,N,Norm,Norm,SBrkr,248,TA,TA,Wd Sdng,Wd Sdng,None,None,0,BrkTil,1,Mod,180.0,1.0,Fa,Unf,Fa,Detchd,1917.0,1224,1,GasA,Gd,1.5Fin,1556,1,NaN,Lvl,Gtl,10632,Inside,72.000000,IR1,0,50,RL,0.0,None,None,0,1,ClearCr,0,3,5,N,0,None,CompShg,Gable,Normal,COD,0,Pave,6,689.0,AllPub,0,1917,1950,2010
96,1212,180,0,None,3,2fmCon,TA,No,1026.0,0.0,Rec,Unf,1.0,0.0,TA,186.0,N,Norm,Norm,SBrkr,168,Fa,Fa,Wd Sdng,Wd Shng,None,None,0,BrkTil,1,Typ,0.0,0.0,None,None,None,None,NaN,1392,0,GasA,TA,1.5Fin,1557,1,TA,Lvl,Gtl,9900,Inside,60.000000,Reg,0,190,RL,0.0,None,None,0,2,SWISU,0,4,5,N,0,None,CompShg,Gable,Normal,ConLD,0,Pave,6,1212.0,AllPub,0,1915,1950,2010
98,942,942,0,None,4,1Fam,TA,No,73.0,0.0,Rec,Unf,0.0,0.0,TA,634.0,N,Norm,Norm,SBrkr,239,TA,TA,Wd Sdng,Stucco,None,None,0,CBlock,1,Typ,0.0,0.0,None,None,None,None,NaN,1884,1,GasW,TA,2Story,1559,1,TA,Lvl,Gtl,6449,Inside,6300.000000,IR1,0,70,C (all),0.0,None,None,0,3,SWISU,0,1,4,N,0,None,CompShg,Gambrel,Abnorml,WD,0,Pave,7,707.0,AllPub,0,1907,1950,2010
100,1832,0,0,None,4,Duplex,TA,No,704.0,0.0,ALQ,Unf,2.0,0.0,TA,1128.0,N,Norm,Norm,SBrkr,0,Fa,Fa,Plywood,Plywood,None,None,0,CBlock,2,Typ,0.0,0.0,None,None,None,None,NaN,1832,0,GasA,TA,1Story,1561,2,TA,Lvl,Gtl,10773,Inside,72.000000,Reg,0,90,RL,72.0,BrkFace,None,0,5,Sawyer,58,3,4,Y,0,None,Tar&Grv,Gable,Normal,WD,0,Pave,8,1832.0,AllPub,0,1967,1967,2010
130,960,0,0,None,3,1Fam,TA,No,0.0,0.0,Unf,Unf,0.0,0.0,TA,960.0,Y,Norm,Norm,SBrkr,0,TA,TA,WdShing,Wd Shng,MnPrv,None,0,CBlock,1,Typ,0.0,0.0,None,None,None,None,NaN,960,0,GasA,Ex,1Story,1591,1,TA,HLS,Gtl,6410,Inside,64.000000,Reg,0,20,RL,0.0,None,None,0,5,Edwards,0,5,4,Y,0,None,CompShg,Hip,Normal,WD,0,Pave,5,960.0,AllPub,0,1958,1958,2010
133,862,1788,0,Pave,6,D

In [26]:
test['MasVnrArea'] = test['MasVnrArea'].replace(np.nan, 0)
test['GarageYrBlt'] = test['GarageYrBlt'].replace(np.nan, 0)
test['GarageCars'] = test['GarageCars'].replace(np.nan, 0)
test['GarageArea'] = test['GarageArea'].replace(np.nan, 0)
test['Exterior1st'] = test['Exterior1st'].replace(np.nan, 'None') #not sure 
test['Exterior2nd'] = test['Exterior2nd'].replace(np.nan, 'None') #not sure 
test['MSZoning'] = test['MSZoning'].replace(np.nan, 'None') #not sure
test['Functional'] = test['Functional'].replace(np.nan, 'Typ')
test['BsmtFullBath'] = test['BsmtFullBath'].replace(np.nan, 0)
test['BsmtHalfBath'] = test['BsmtHalfBath'].replace(np.nan, 0)
test['BsmtFinSF1'] = test['BsmtFinSF1'].replace(np.nan, 0)
test['BsmtFinSF2'] = test['BsmtFinSF2'].replace(np.nan, 0)
test['BsmtUnfSF'] = test['BsmtUnfSF'].replace(np.nan, 0)
test['TotalBsmtSF'] = test['TotalBsmtSF'].replace(np.nan, 0)
test['KitchenQual'] = test['KitchenQual'].replace(np.nan, 'TA')
test['SaleType'] = test['SaleType'].replace(np.nan, 'WD') #most frequent 
test['Utilities'] = test['Utilities'].fillna('AllPub')

In [29]:
test = pd.get_dummies(test, drop_first=True)